<h2> 3.6 Featurizing text data with tfidf weighted word-vectors </h2>

In [12]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import time
import warnings
import numpy as np
from nltk.corpus import stopwords
from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
warnings.filterwarnings("ignore")
import sys
import os 
import pandas as pd
import numpy as np
from tqdm import tqdm

# exctract word2vec vectors
# https://github.com/explosion/spaCy/issues/1721
# http://landinghub.visualstudio.com/visual-cpp-build-tools
import spacy
nlp = spacy.load('en_core_web_sm')

In [4]:
# avoid decoding problems
df = pd.read_csv("train.csv")
 
# encode questions to unicode
# https://stackoverflow.com/a/6812069
# ----------------- python 2 ---------------------
# df['question1'] = df['question1'].apply(lambda x: unicode(str(x),"utf-8"))
# df['question2'] = df['question2'].apply(lambda x: unicode(str(x),"utf-8"))
# ----------------- python 3 ---------------------
df['question1'] = df['question1'].apply(lambda x: str(x))
df['question2'] = df['question2'].apply(lambda x: str(x))

In [5]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
# merge texts
questions = list(df['question1']) + list(df['question2'])

tfidf = TfidfVectorizer(lowercase=False, )
tfidf.fit_transform(questions)

# dict key:word and value:tf-idf score
#word2tfidf = dict(zip(tfidf.get_feature_names(), tfidf.idf_))
word2tfidf = dict(zip(tfidf.get_feature_names_out(), tfidf.idf_))

- After we find TF-IDF scores, we convert each question to a weighted average of word2vec vectors by these scores.
- here we use a pre-trained GLOVE model which comes free with "Spacy".  https://spacy.io/usage/vectors-similarity
- It is trained on Wikipedia and therefore, it is stronger in terms of word semantics. 

In [13]:
# en_vectors_web_lg, which includes over 1 million unique vectors.
#nlp = spacy.load('en_core_web_sm')
vecs1 = []
# https://github.com/noamraph/tqdm
# tqdm is used to print the progress bar
for qu1 in tqdm(list(df['question1'])):
    doc1 = nlp(qu1) 
    # 384 is the number of dimensions of vectors 
    mean_vec1 = np.zeros([len(doc1), len(doc1[0].vector)])
    for word1 in doc1:
        # word2vec
        vec1 = word1.vector
        # fetch df score
        try:
            idf = word2tfidf[str(word1)]
        except:
            idf = 0
        # compute final vec
        mean_vec1 += vec1 * idf
    mean_vec1 = mean_vec1.mean(axis=0)
    vecs1.append(mean_vec1)
df['q1_feats_m'] = list(vecs1)


100%|██████████████████████████████████| 404290/404290 [21:09<00:00, 318.51it/s]


In [15]:
vecs2 = []
for qu2 in tqdm(list(df['question2'])):
    doc2 = nlp(qu2) 
    mean_vec2 = np.zeros([len(doc1), len(doc2[0].vector)])
    for word2 in doc2:
        # word2vec
        vec2 = word2.vector
        # fetch df score
        try:
            idf = word2tfidf[str(word2)]
        except:
            #print word
            idf = 0
        # compute final vec
        mean_vec2 += vec2 * idf
    mean_vec2 = mean_vec2.mean(axis=0)
    vecs2.append(mean_vec2)
df['q2_feats_m'] = list(vecs2)

100%|██████████████████████████████████| 404290/404290 [20:46<00:00, 324.23it/s]


In [16]:
#prepro_features_train.csv (Simple Preprocessing Feartures)
#nlp_features_train.csv (NLP Features)
if os.path.isfile('nlp_features_train.csv'):
    dfnlp = pd.read_csv("nlp_features_train.csv",encoding='latin-1')
else:
    print("download nlp_features_train.csv from drive or run previous notebook")

if os.path.isfile('basic_features_df.csv'):
    dfppro = pd.read_csv("basic_features_df.csv",encoding='latin-1')
else:
    print("download basic_features_df.csv")

In [17]:
df1 = dfnlp.drop(['qid1','qid2','question1','question2'],axis=1)
df2 = dfppro.drop(['qid1','qid2','question1','question2','is_duplicate'],axis=1)
df3 = df.drop(['qid1','qid2','question1','question2','is_duplicate'],axis=1)
df3_q1 = pd.DataFrame(df3.q1_feats_m.values.tolist(), index= df3.index)
df3_q2 = pd.DataFrame(df3.q2_feats_m.values.tolist(), index= df3.index)

In [18]:
# dataframe of nlp features
df1.head()

,id,is_duplicate,cwc_min,cwc_max,csc_min,csc_max,ctc_min,ctc_max,last_word_eq,first_word_eq,abs_len_diff,mean_len,token_set_ratio,token_sort_ratio,fuzz_ratio,fuzz_partial_ratio,longest_substr_ratio
0,0,0,0.999980,0.833319,0.999983,0.999983,0.916659,0.785709,0.0,1.0,2.0,13.0,100,93,93,100,0.982759
1,1,0,0.799984,0.399996,0.749981,0.599988,0.699993,0.466664,0.0,1.0,5.0,12.5,86,63,66,75,0.596154
2,2,0,0.399992,0.333328,0.399992,0.249997,0.399996,0.285712,0.0,1.0,4.0,12.0,66,66,54,54,0.166667
3,3,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,2.0,12.0,36,36,35,40,0.039216
4,4,0,0.399992,0.199998,0.999950,0.666644,0.571420,0.307690,0.0,1.0,6.0,10.0,67,47,46,56,0.175000


In [19]:
# data before preprocessing 
df2.head()

,id,freq_qid1,freq_qid2,q1len,q2len,q1_n_words,q2_n_words,word_Common,word_Total,word_share,freq_q1+freq_q2,freq_q1-freq_q2
0,0,1,1,66,57,14,12,10,26,0.384615,2,0
1,1,4,1,51,88,8,13,4,21,0.190476,5,3
2,2,1,1,73,59,14,10,4,24,0.166667,2,0
3,3,1,1,50,65,11,9,0,20,0.000000,2,0
4,4,3,1,76,39,13,7,2,20,0.100000,4,2


In [20]:
# Questions 1 tfidf weighted word2vec
df3_q1.head()

,0,1,2,3,4,5,6,7,8,9,...,86,87,88,89,90,91,92,93,94,95
0,16.463352,-45.934097,9.832652,-19.447236,16.256207,-11.408635,45.781356,33.793397,-0.682303,10.741591,...,10.027325,-33.372708,-48.610676,23.520856,-20.693879,9.707473,44.017381,68.670245,-17.080290,33.657352
1,-15.513317,-26.773427,22.476613,38.939707,31.664148,11.493577,18.258096,56.702168,3.560232,-46.756694,...,-6.072493,23.089776,-38.483045,35.745371,-14.073073,7.308293,89.132794,3.295310,0.226583,88.639372
2,-17.676360,-9.333407,-9.743487,-9.513847,9.613598,-14.764669,23.965153,13.467981,-18.925067,2.744790,...,28.016301,-0.865154,-24.250927,15.859982,-19.880625,-20.721361,28.209778,41.850922,-17.749369,30.532825
3,61.410197,-18.528254,-11.641802,-12.944753,-25.763726,-20.467977,12.122646,10.075657,16.520064,-7.103114,...,-16.086371,-1.919420,3.626949,42.526903,48.064339,-28.988919,29.753618,27.724041,1.117192,15.843501
4,-16.935536,-43.565436,-0.564589,-44.367042,-40.639746,15.802763,47.420121,79.964160,12.941927,-13.307759,...,-12.924427,-15.566456,-51.009174,103.080692,-26.997214,34.913615,89.270826,76.832048,5.808152,89.445565


In [21]:
# Questions 2 tfidf weighted word2vec
df3_q2.head()

,0,1,2,3,4,5,6,7,8,9,...,86,87,88,89,90,91,92,93,94,95
0,17.591981,-45.190803,5.264228,-16.323091,8.937822,-9.329011,39.247609,41.735724,-10.110281,13.893752,...,12.478971,-30.068917,-47.913704,14.714810,-17.241804,6.486739,26.456939,63.701757,-17.887831,21.516734
1,-22.684537,-72.190472,-6.707389,36.345250,8.030899,-5.487787,8.623319,36.662627,-7.383569,-39.861671,...,4.830289,19.611325,-35.900326,45.733025,-2.857657,5.916825,82.456656,34.427449,-0.153893,83.436965
2,5.474999,-16.175595,-6.273923,1.055976,-22.802805,-1.355452,16.201954,32.944558,-4.630627,6.233710,...,51.079289,-11.944448,-51.070296,12.397100,-19.068655,-5.625120,48.529718,48.813845,-6.016332,10.582182
3,19.534101,-15.777414,-6.863121,4.812231,-29.383096,-6.756095,-7.505566,0.548628,-17.597091,-14.904704,...,22.016938,8.856647,-6.845016,22.816602,0.418038,-18.978343,9.303697,41.359699,4.616411,32.863474
4,-10.295483,-31.240734,10.990936,0.656095,-23.303608,-3.377758,30.301446,16.654260,-0.161822,-3.671971,...,-3.956319,-13.591864,-26.024700,13.573336,-10.809351,-2.419427,35.918887,36.435140,-10.428506,16.641608


In [22]:
print("Number of features in nlp dataframe :", df1.shape[1])
print("Number of features in preprocessed dataframe :", df2.shape[1])
print("Number of features in question1 w2v  dataframe :", df3_q1.shape[1])
print("Number of features in question2 w2v  dataframe :", df3_q2.shape[1])
print("Number of features in final dataframe  :", df1.shape[1]+df2.shape[1]+df3_q1.shape[1]+df3_q2.shape[1])

Number of features in nlp dataframe : 17
Number of features in preprocessed dataframe : 12
Number of features in question1 w2v  dataframe : 96
Number of features in question2 w2v  dataframe : 96
Number of features in final dataframe  : 221


In [23]:
# storing the final features to csv file
if not os.path.isfile('final_features.csv'):
    df3_q1['id']=df1['id']
    df3_q2['id']=df1['id']
    df1  = df1.merge(df2, on='id',how='left')
    df2  = df3_q1.merge(df3_q2, on='id',how='left')
    result  = df1.merge(df2, on='id',how='left')
    result.to_csv('final_features.csv')